Tutorial basado en:

https://huggingface.co/blog/sentiment-analysis-python

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

In [ ]:
!pip install xformers

#**1 - Uso de modelos predeterminados pre-entrenados:**

Hat modelos pre-entrenados que puedes usar de manera directa, en particular para análisis de sentimiento:

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")

data = ["I love you", "I hate you"]

sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

#**2 - Fine-tuninig model**

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install datasets transformers

Preprocess data

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Training the model:

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.we

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)

   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]

   return {"accuracy": accuracy, "f1": f1}


In [ ]:
!pip install transformers[torch]   # instala y reinicia el entorno de ejecución...

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()   # genera un token que sea de escritura (write):

In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


Cloning https://huggingface.co/leFalcon/finetuning-sentiment-model-3000-samples into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/255M [00:00<?, ?B/s]

Download file runs/Jun13_23-44-59_ec228bbe3f8e/events.out.tfevents.1686700547.ec228bbe3f8e.1792.1: 100%|######…

Download file training_args.bin: 100%|##########| 3.87k/3.87k [00:00<?, ?B/s]

Download file runs/Jun13_23-44-59_ec228bbe3f8e/events.out.tfevents.1686699990.ec228bbe3f8e.1792.0: 100%|######…

Clean file runs/Jun13_23-44-59_ec228bbe3f8e/events.out.tfevents.1686700547.ec228bbe3f8e.1792.1: 100%|#########…

Clean file training_args.bin:  26%|##5       | 1.00k/3.87k [00:00<?, ?B/s]

Clean file runs/Jun13_23-44-59_ec228bbe3f8e/events.out.tfevents.1686699990.ec228bbe3f8e.1792.0:  24%|##3      …

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [ ]:
trainer.train()   # entrenamiento con fine-tuning...  tarda como 5.5 mins con el GPU de Colab..

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=376, training_loss=0.30798037508700754, metrics={'train_runtime': 326.0554, 'train_samples_per_second': 18.402, 'train_steps_per_second': 1.153, 'total_flos': 783875831546880.0, 'train_loss': 0.30798037508700754, 'epoch': 2.0})

In [ ]:
trainer.evaluate()   # evaluemos el desempeño de este modelo...


<ipython-input-10-6cdd98e0f597>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.2834215760231018,
 'eval_accuracy': 0.8766666666666667,
 'eval_f1': 0.877887788778878,
 'eval_runtime': 7.0397,
 'eval_samples_per_second': 42.615,
 'eval_steps_per_second': 2.699,
 'epoch': 2.0}

Obtenemos un 87% para este subconjunto de comentarios de iMDB

###Evaluemos el modelo con datos nuevos...

In [ ]:
!apt-get install git-lfs   # pero primero debemos instalar git-lfs para usar git en el repositorio de nuestro modelo...


Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


In [ ]:
trainer.push_to_hub()    #  caeguemos primero nuestro modelo al Hub...

###y listo, podemos aplicar nuestro modelo ajustado (fine-tuned) con estos nuevos comentarios...

In [ ]:
from transformers import pipeline

In [ ]:
 sentiment_model = pipeline(model="federicopascual/finetuning-sentiment-model-3000-samples")

sentiment_model(["I love this move", "This movie sucks!"])

[{'label': 'LABEL_1', 'score': 0.9558863043785095},
 {'label': 'LABEL_0', 'score': 0.9413502216339111}]